In [0]:
-- En este notebook se desarrolla el proceso de limpieza, transformación y preparación de datos utilizando SQL,
-- aplicado a un dataset de consumo energético diario de tiendas físicas.
--
-- El objetivo es demostrar cómo implementar un flujo ETL claro y estructurado utilizando:
--  - filtros previos para limpieza de datos,
--  - imputación de valores nulos mediante funciones analíticas,
--  - creación de columnas derivadas,
--  - validación de métricas operativas.
--
-- El proyecto pone especial énfasis en el uso correcto de WHERE, funciones de ventana
-- y vistas temporales para mantener consultas legibles, mantenibles y alineadas
-- con buenas prácticas de SQL en entornos analíticos.


-- PASO 1: CÁLCULO DE PROMEDIOS OPERATIVOS
-- Se calculan los valores promedio de área, consumo y demanda únicamente para tiendas activas,
-- agrupando por país y tipo de tienda.
-- Estos promedios servirán como base para la imputación de valores nulos posteriormente.

CREATE OR REPLACE TABLE promedios AS
SELECT
    pais,
    tipo_tienda,
    AVG(area_m2)       AS promedio_area_m2,
    AVG(consumo_kwh)   AS promedio_consumo_kwh,
    AVG(demanda_kw)    AS promedio_demanda_kw
FROM workspace.default.consumo_energia_tiendas_diario_2023_2024
WHERE estado_tienda = 'Activa'
GROUP BY
    pais,
    tipo_tienda;


-- PASO 2: FILTRADO DE TIENDAS ACTIVAS Y UNIÓN CON PROMEDIOS
-- Se crea una vista temporal que contiene únicamente tiendas activas,
-- incorporando los promedios calculados por país y tipo de tienda.
-- El LEFT JOIN garantiza que se conserven todos los registros activos.

CREATE OR REPLACE TEMP VIEW df_active AS
SELECT
    *
FROM workspace.default.consumo_energia_tiendas_diario_2023_2024
LEFT JOIN promedios
    USING (pais, tipo_tienda)
WHERE estado_tienda = 'Activa';


-- PASO 3: IMPUTACIÓN DE NULOS Y CREACIÓN DE MÉTRICAS DERIVADAS
-- Se imputan valores faltantes utilizando COALESCE, priorizando el valor original
-- y utilizando el promedio correspondiente cuando el dato es nulo.
--
-- Además, se calculan indicadores de eficiencia energética normalizados por área,
-- permitiendo comparaciones homogéneas entre tiendas de distinto tamaño.

CREATE OR REPLACE TEMP VIEW df AS
SELECT
    tienda_id,
    tipo_tienda,
    pais,
    region,

    COALESCE(area_m2, promedio_area_m2)         AS area_m2,
    COALESCE(consumo_kwh, promedio_consumo_kwh) AS consumo_kwh,
    COALESCE(demanda_kw, promedio_demanda_kw)   AS demanda_kw,

    tarifa,
    mes,
    anio,
    estado_tienda,

    consumo_kwh / area_m2   AS kpi_kwh_m2,
    demanda_kw  / area_m2   AS kpi_kw_m2
FROM df_active;


-- PASO 4: VALIDACIÓN FINAL
-- Se consulta el dataset resultante para validación visual
-- y comprobación de consistencia tras la imputación y creación de KPIs.

SELECT *
FROM df;
